In [1]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [7]:
doc = nlp("I am looking forward to have a dinner with Saurav today.")

In [8]:
for token in doc:
    print(token.text + ' - ' + token.dep_)

I - nsubj
am - aux
looking - ROOT
forward - advmod
to - aux
have - advcl
a - det
dinner - dobj
with - prep
Saurav - pobj
today - npadvmod
. - punct


In [9]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
          [child for child in token.children])

I nsubj looking VERB []
am aux looking VERB []
looking ROOT looking VERB [I, am, forward, have, .]
forward advmod looking VERB []
to aux have VERB []
have advcl looking VERB [to, dinner, today]
a det dinner NOUN []
dinner dobj have VERB [a, with]
with prep dinner NOUN [Saurav]
Saurav pobj with ADP []
today npadvmod have VERB []
. punct looking VERB []


In [1]:
import keras

Using TensorFlow backend.


In [3]:
# -*- coding: utf-8 -*-
'''An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits reversed:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits reversed:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits reversed:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_2 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_____________________________

In [13]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """

tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]
print(encoded)

[2, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 2, 14, 15, 1, 16, 17, 18, 1, 3, 19, 20, 21]


In [14]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

22


In [20]:
sequences = list()
for i in range(1, len(encoded)):
    sequence = encoded[i - 1 : i + 1]
    print(sequence)
    sequences.append(sequence)

print(len(sequences))

[2, 1]
[1, 3]
[3, 4]
[4, 5]
[5, 6]
[6, 7]
[7, 8]
[8, 9]
[9, 10]
[10, 11]
[11, 12]
[12, 13]
[13, 2]
[2, 14]
[14, 15]
[15, 1]
[1, 16]
[16, 17]
[17, 18]
[18, 1]
[1, 3]
[3, 19]
[19, 20]
[20, 21]
24


In [23]:
sequences = array(sequences)
x, y = sequences[:, 0], sequences[:, 1]

In [24]:
y = to_categorical(y, num_classes=vocab_size)
print(y)

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [28]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1, 10)             220       
_________________________________________________________________
lstm_9 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_7 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=500, verbose=2)

Epoch 1/500
 - 1s - loss: 3.0907 - acc: 0.0417
Epoch 2/500
 - 0s - loss: 3.0901 - acc: 0.0833
Epoch 3/500
 - 0s - loss: 3.0894 - acc: 0.0833
Epoch 4/500
 - 0s - loss: 3.0886 - acc: 0.0833
Epoch 5/500
 - 0s - loss: 3.0878 - acc: 0.0833
Epoch 6/500
 - 0s - loss: 3.0870 - acc: 0.1250
Epoch 7/500
 - 0s - loss: 3.0862 - acc: 0.1250
Epoch 8/500
 - 0s - loss: 3.0853 - acc: 0.1250
Epoch 9/500
 - 0s - loss: 3.0845 - acc: 0.1250
Epoch 10/500
 - 0s - loss: 3.0836 - acc: 0.1250
Epoch 11/500
 - 0s - loss: 3.0828 - acc: 0.1250
Epoch 12/500
 - 0s - loss: 3.0819 - acc: 0.1250
Epoch 13/500
 - 0s - loss: 3.0810 - acc: 0.1250
Epoch 14/500
 - 0s - loss: 3.0801 - acc: 0.1250
Epoch 15/500
 - 0s - loss: 3.0792 - acc: 0.1250
Epoch 16/500
 - 0s - loss: 3.0783 - acc: 0.1250
Epoch 17/500
 - 0s - loss: 3.0773 - acc: 0.1250
Epoch 18/500
 - 0s - loss: 3.0763 - acc: 0.1250
Epoch 19/500
 - 0s - loss: 3.0753 - acc: 0.1250
Epoch 20/500
 - 0s - loss: 3.0743 - acc: 0.1250
Epoch 21/500
 - 0s - loss: 3.0733 - acc: 0.1250
E

Epoch 171/500
 - 0s - loss: 1.8844 - acc: 0.5000
Epoch 172/500
 - 0s - loss: 1.8692 - acc: 0.5417
Epoch 173/500
 - 0s - loss: 1.8540 - acc: 0.5417
Epoch 174/500
 - 0s - loss: 1.8388 - acc: 0.5417
Epoch 175/500
 - 0s - loss: 1.8236 - acc: 0.5417
Epoch 176/500
 - 0s - loss: 1.8084 - acc: 0.5417
Epoch 177/500
 - 0s - loss: 1.7932 - acc: 0.5417
Epoch 178/500
 - 0s - loss: 1.7780 - acc: 0.5417
Epoch 179/500
 - 0s - loss: 1.7629 - acc: 0.5417
Epoch 180/500
 - 0s - loss: 1.7477 - acc: 0.5417
Epoch 181/500
 - 0s - loss: 1.7326 - acc: 0.5833
Epoch 182/500
 - 0s - loss: 1.7175 - acc: 0.5833
Epoch 183/500
 - 0s - loss: 1.7024 - acc: 0.6250
Epoch 184/500
 - 0s - loss: 1.6873 - acc: 0.6250
Epoch 185/500
 - 0s - loss: 1.6723 - acc: 0.6250
Epoch 186/500
 - 0s - loss: 1.6573 - acc: 0.6250
Epoch 187/500
 - 0s - loss: 1.6423 - acc: 0.7083
Epoch 188/500
 - 0s - loss: 1.6273 - acc: 0.7083
Epoch 189/500
 - 0s - loss: 1.6124 - acc: 0.7083
Epoch 190/500
 - 0s - loss: 1.5974 - acc: 0.7500
Epoch 191/500
 - 0s 

 - 0s - loss: 0.3580 - acc: 0.8750
Epoch 339/500
 - 0s - loss: 0.3558 - acc: 0.8750
Epoch 340/500
 - 0s - loss: 0.3537 - acc: 0.8750
Epoch 341/500
 - 0s - loss: 0.3516 - acc: 0.8750
Epoch 342/500
 - 0s - loss: 0.3495 - acc: 0.8750
Epoch 343/500
 - 0s - loss: 0.3474 - acc: 0.8750
Epoch 344/500
 - 0s - loss: 0.3454 - acc: 0.8750
Epoch 345/500
 - 0s - loss: 0.3435 - acc: 0.8750
Epoch 346/500
 - 0s - loss: 0.3415 - acc: 0.8750
Epoch 347/500
 - 0s - loss: 0.3396 - acc: 0.8750
Epoch 348/500
 - 0s - loss: 0.3378 - acc: 0.8750
Epoch 349/500
 - 0s - loss: 0.3359 - acc: 0.8750
Epoch 350/500
 - 0s - loss: 0.3341 - acc: 0.8750
Epoch 351/500
 - 0s - loss: 0.3324 - acc: 0.8750
Epoch 352/500
 - 0s - loss: 0.3306 - acc: 0.8750
Epoch 353/500
 - 0s - loss: 0.3289 - acc: 0.8750
Epoch 354/500
 - 0s - loss: 0.3272 - acc: 0.8750
Epoch 355/500
 - 0s - loss: 0.3256 - acc: 0.8750
Epoch 356/500
 - 0s - loss: 0.3240 - acc: 0.8750
Epoch 357/500
 - 0s - loss: 0.3224 - acc: 0.8750
Epoch 358/500
 - 0s - loss: 0.3208

In [41]:
in_text = 'water'
print(in_text)
encoded = tokenizer.texts_to_sequences([in_text])[0]
encoded = array(encoded)
yhat = model.predict_classes(encoded, verbose=0)
for word, index in tokenizer.word_index.items():
    if index == yhat:
        print(word)

water
jack
